In [17]:
import requests as r
import urllib3
import json
import time

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

base_url = "https://9.155.108.201/scalemgmt/v2"
auth_data = ("admin","admin001")


headers = {'content-type' : 'application/json', 'accepts' : 'application/json'}

def printCurl(method, url, data = None):
    if data is None:
        print "curl -X {} -u {}:{} -k -H 'content-type: application/json' -H 'accepts: application/json' '{}'" \
                .format(method, auth_data[0], auth_data[1], url)    
    else:
        print "curl -X {} -u {}:{} -k -H 'content-type: application/json' -H 'accepts: application/json' '{}' --data '{}'" \
                .format(method, auth_data[0], auth_data[1], url, json.dumps(data))

def printJson(res):
    for line in res.text.split("\n"):
        print line
    #output = json.dumps(res.json(), sort_keys=True, indent=4)
    #for line in output.split("\n"):
    #    print line
        
def get(url, params=None,printResult=True, printRequest=True):
    if printRequest:
        printCurl("GET", base_url + url)
    res = r.get(base_url + url, auth=auth_data, verify=False, headers=headers, params=params)
    if printResult:
        printJson(res)
    return res

def delete(url, printResult=True, printRequest=True):
    if printRequest:
        printCurl("DELETE", base_url + url)
    res = r.delete(base_url + url, auth=auth_data, verify=False, headers=headers)
    if printResult:
        printJson(res)
    return res
    
def post(url, data, printResult=True, printRequest=True):
    if printRequest:
        printCurl("POST", base_url + url, data)
    res = r.post(base_url + url, auth=auth_data, verify=False, json=data, headers=headers)
    if printResult:
        printJson(res)
    return res

def put(url, data, printResult=True, printRequest=True):
    if printRequest:
        printCurl("PUT", base_url + url, data)
    res = r.put(base_url + url, auth=auth_data, verify=False, json=data, headers=headers)
    if printResult:
        printJson(res)
    return res

def getJobId(res):
    return str(res.json()['jobs'][0]['jobId'])
    
def waitForJob(jobId, maxRetries=10):
    completed = False
    retries = 0
    status = ""
    while not completed and retries < maxRetries:
        time.sleep(5)
        res = get("/jobs/" + str(jobId), printResult=False)
        status = str(res.json()['jobs'][0]['completed'])
        completed = status != "N/A"
        if not completed:
            print "Job not yet completed, status: " + status
        retries += 1
    return (completed, res)

def waitForJobResult(res, maxRetries=10):
    jobId = getJobId(res)
    (completed, res) = waitForJob(jobId, maxRetries)
    printJson(res)
    return (completed, res)

    

get("/info")

curl -X GET -u admin:admin001 -k -H 'content-type: application/json' -H 'accepts: application/json' 'https://9.155.108.201/scalemgmt/v2/info'
{
  "info" : {
    "name" : "Spectrum Scale REST Management",
    "restApiVersion" : "2.2.0",
    "serverVersion" : "5.0.3-0.190131.054656",
    "paths" : {
      "/access" : [ "POST", "GET" ],
      "/access/status" : [ "GET" ],
      "/ces/addresses" : [ "GET" ],
      "/ces/addresses/{cesAddress}" : [ "GET" ],
      "/ces/services" : [ "GET" ],
      "/ces/services/{service}" : [ "GET" ],
      "/cliauditlog" : [ "GET" ],
      "/cluster" : [ "GET" ],
      "/cluster/jobs" : [ "GET" ],
      "/cluster/jobs/{jobId}" : [ "GET", "DELETE" ],
      "/components/elements" : [ "GET" ],
      "/components/elements/{componentId}" : [ "GET" ],
      "/components/specifications" : [ "GET" ],
      "/config" : [ "PUT", "GET" ],
      "/filesystems" : [ "GET" ],
      "/filesystems/{filesystemName}" : [ "GET" ],
      "/filesystems/{filesystemName}/acl/{pa

<Response [200]>

# Step 1: Create a fileset

In [18]:
filesetName = "newResearchFset"

data = {
        "filesetName": filesetName,
        "inodeSpace": "root"
    }
waitForJobResult(post("/filesystems/gpfs0/filesets", data))
#waitForJobResult(delete("/filesystems/gpfs0/filesets/" + filesetName))

curl -X POST -u admin:admin001 -k -H 'content-type: application/json' -H 'accepts: application/json' 'https://9.155.108.201/scalemgmt/v2/filesystems/gpfs0/filesets' --data '{"filesetName": "newResearchFset", "inodeSpace": "root"}'
{
  "jobs" : [ {
    "jobId" : 1000000000036,
    "status" : "RUNNING",
    "submitted" : "2019-03-21 09:14:27,832",
    "completed" : "N/A",
    "runtime" : 6,
    "request" : {
      "data" : {
        "filesetName" : "newResearchFset",
        "inodeSpace" : "root"
      },
      "type" : "POST",
      "url" : "/scalemgmt/v2/filesystems/gpfs0/filesets"
    },
    "result" : { },
    "pids" : [ ]
  } ],
  "status" : {
    "code" : 202,
    "message" : "The request was accepted for processing."
  }
}
curl -X GET -u admin:admin001 -k -H 'content-type: application/json' -H 'accepts: application/json' 'https://9.155.108.201/scalemgmt/v2/jobs/1000000000036'
{
  "jobs" : [ {
    "jobId" : 1000000000036,
    "status" : "COMPLETED",
    "submitted" : "2019-03-21 09

(True, <Response [200]>)

In [19]:
get("/filesystems/gpfs0/filesets/" + filesetName)

curl -X GET -u admin:admin001 -k -H 'content-type: application/json' -H 'accepts: application/json' 'https://9.155.108.201/scalemgmt/v2/filesystems/gpfs0/filesets/newResearchFset'
{
  "filesets" : [ {
    "config" : {
      "comment" : "",
      "created" : "2019-03-21 09:14:29,000",
      "iamMode" : "off",
      "id" : 3,
      "inodeSpace" : 0,
      "inodeSpaceMask" : 0,
      "isInodeSpaceOwner" : false,
      "maxNumInodes" : 65792,
      "oid" : 8,
      "parentId" : 0,
      "path" : "/mnt/gpfs0/newResearchFset",
      "permissionChangeMode" : "chmodAndSetacl",
      "rootInode" : 7425,
      "snapId" : 0,
      "status" : "Linked"
    },
    "filesetName" : "newResearchFset",
    "filesystemName" : "gpfs0",
    "usage" : {
      "allocatedInodes" : 65792,
      "inodeSpaceFreeInodes" : 61754,
      "inodeSpaceUsedInodes" : 4038,
      "usedBytes" : 0,
      "usedInodes" : 2
    }
  } ],
  "status" : {
    "code" : 200,
    "message" : "The request finished successfully."
  }
}

<Response [200]>

# Step 2: Set a quota

In [20]:
data = {
  "operationType": "setQuota",
  "quotaType": "FILESET",
  "objectName": filesetName,
  "blockSoftLimit": "1G",
  "blockHardLimit": "2G",
  "filesSoftLimit": "1M",
  "filesHardLimit": "2M",
  "filesGracePeriod": "5hour",
  "blockGracePeriod": "2days"
}
waitForJobResult(post("/filesystems/gpfs0/quotas", data))


curl -X POST -u admin:admin001 -k -H 'content-type: application/json' -H 'accepts: application/json' 'https://9.155.108.201/scalemgmt/v2/filesystems/gpfs0/quotas' --data '{"filesGracePeriod": "5hour", "filesSoftLimit": "1M", "objectName": "newResearchFset", "operationType": "setQuota", "blockSoftLimit": "1G", "quotaType": "FILESET", "blockGracePeriod": "2days", "filesHardLimit": "2M", "blockHardLimit": "2G"}'
{
  "jobs" : [ {
    "jobId" : 1000000000037,
    "status" : "RUNNING",
    "submitted" : "2019-03-21 09:18:08,128",
    "completed" : "N/A",
    "runtime" : 0,
    "request" : {
      "data" : {
        "blockGracePeriod" : "2days",
        "blockHardLimit" : "2G",
        "blockSoftLimit" : "1G",
        "filesGracePeriod" : "5hour",
        "filesHardLimit" : "2M",
        "filesSoftLimit" : "1M",
        "objectName" : "newResearchFset",
        "operationType" : "setQuota",
        "quotaType" : "FILESET"
      },
      "type" : "POST",
      "url" : "/scalemgmt/v2/filesystem

(True, <Response [200]>)

In [21]:
get("/filesystems/gpfs0/quotas")

curl -X GET -u admin:admin001 -k -H 'content-type: application/json' -H 'accepts: application/json' 'https://9.155.108.201/scalemgmt/v2/filesystems/gpfs0/quotas'
{
  "quotas" : [ {
    "blockGrace" : "none",
    "blockInDoubt" : 0,
    "blockLimit" : 2097152,
    "blockQuota" : 1048576,
    "blockUsage" : 0,
    "filesGrace" : "none",
    "filesInDoubt" : 0,
    "filesLimit" : 2097152,
    "filesQuota" : 1048576,
    "filesUsage" : 1,
    "filesystemName" : "gpfs0",
    "isDefaultQuota" : false,
    "objectId" : 3,
    "objectName" : "newResearchFset",
    "quotaId" : 24,
    "quotaType" : "FILESET"
  } ],
  "status" : {
    "code" : 200,
    "message" : "The request finished successfully."
  }
}


<Response [200]>

# Step 3: Create a SMB share

In [22]:
shareName = filesetName + "SMBShare"

data = {
  "shareName": shareName,
  "path": "/mnt/gpfs0/" + filesetName,
  "smbOptions": {
    "browseable": "yes",
    "smbEncrypt": "auto",
    "comment": "This share is used for a new research project. Ticket ID #123456",
    "cscPolicy": "manual",
    "fileIdAlgorithm": "fsname",
    "gpfsLeases": "yes",
    "gpfsRecalls": "yes",
    "gpfsShareModes": "yes",
    "gpfsSyncIo": "yes",
    "hideUnreadable": "yes",
    "opLocks": "yes",
    "posixLocking": "yes",
    "readOnly": "yes",
    "syncOpsOnClose": "yes",
    "hideDotFiles": "yes"
  }
}

waitForJobResult(post("/smb/shares", data))
#waitForJobResult(delete("/smb/shares/" + shareName))



curl -X POST -u admin:admin001 -k -H 'content-type: application/json' -H 'accepts: application/json' 'https://9.155.108.201/scalemgmt/v2/smb/shares' --data '{"shareName": "newResearchFsetSMBShare", "smbOptions": {"comment": "This share is used for a new research project. Ticket ID #123456", "gpfsShareModes": "yes", "cscPolicy": "manual", "gpfsSyncIo": "yes", "hideUnreadable": "yes", "readOnly": "yes", "hideDotFiles": "yes", "fileIdAlgorithm": "fsname", "browseable": "yes", "syncOpsOnClose": "yes", "opLocks": "yes", "posixLocking": "yes", "smbEncrypt": "auto", "gpfsRecalls": "yes", "gpfsLeases": "yes"}, "path": "/mnt/gpfs0/newResearchFset"}'
{
  "jobs" : [ {
    "jobId" : 1000000000038,
    "status" : "RUNNING",
    "submitted" : "2019-03-21 09:19:15,526",
    "completed" : "N/A",
    "runtime" : 1,
    "request" : {
      "data" : {
        "path" : "/mnt/gpfs0/newResearchFset",
        "shareName" : "newResearchFsetSMBShare",
        "smbOptions" : {
          "browseable" : "yes",
  

(True, <Response [200]>)

# Step 4: Create a capacity threshold for the new fileset

In [23]:
rule_name = filesetName + "Capacity"
data = {
  "ruleName": rule_name,
  "userActionWarn": "The capacity of fileset "  + filesetName + " reached the warning level.",
  "userActionError": "The capacity of fileset "  + filesetName + " reached the error level.",
  "metric": "gpfs_rq_blk_current",
  "sensitivity": 300,
  "filterBy": "gpfs_fset_name=" + filesetName,
  "errorLevel": 1945600,
  "warnLevel": 1228800,
  "direction": "high",
  "hysteresis": 5
}
waitForJobResult(post("/thresholds", data))
#waitForJobResult(delete("/thresholds/" + rule_name))

curl -X POST -u admin:admin001 -k -H 'content-type: application/json' -H 'accepts: application/json' 'https://9.155.108.201/scalemgmt/v2/thresholds' --data '{"userActionWarn": "The capacity of fileset newResearchFset reached the warning level.", "direction": "high", "filterBy": "gpfs_fset_name=newResearchFset", "errorLevel": 1945600, "ruleName": "newResearchFsetCapacity", "userActionError": "The capacity of fileset newResearchFset reached the error level.", "metric": "gpfs_rq_blk_current", "hysteresis": 5, "sensitivity": 300, "warnLevel": 1228800}'
{
  "jobs" : [ {
    "jobId" : 1000000000039,
    "status" : "RUNNING",
    "submitted" : "2019-03-21 09:23:49,343",
    "completed" : "N/A",
    "runtime" : 0,
    "request" : {
      "type" : "POST",
      "url" : "/scalemgmt/v2/thresholds"
    },
    "result" : { },
    "pids" : [ ]
  } ],
  "status" : {
    "code" : 202,
    "message" : "The request was accepted for processing."
  }
}
curl -X GET -u admin:admin001 -k -H 'content-type: ap

(True, <Response [200]>)

In [24]:
get("/thresholds/" + rule_name)

curl -X GET -u admin:admin001 -k -H 'content-type: application/json' -H 'accepts: application/json' 'https://9.155.108.201/scalemgmt/v2/thresholds/newResearchFsetCapacity'
{
  "status" : {
    "code" : 200,
    "message" : "The request finished successfully."
  },
  "thresholds" : [ {
    "ruleName" : "newResearchFsetCapacity",
    "frequency" : 300,
    "tags" : "thresholds",
    "userActionWarn" : "The capacity of fileset newResearchFset reached the warning level.",
    "userActionError" : "The capacity of fileset newResearchFset reached the error level.",
    "type" : "metric",
    "metric" : "gpfs_rq_blk_current",
    "metricOp" : "noOperation",
    "sensitivity" : 300,
    "computation" : "None",
    "filterBy" : "gpfs_fset_name=newResearchFset",
    "groupBy" : "None",
    "errorLevel" : 1945600.0,
    "warnLevel" : 1228800.0,
    "direction" : "high",
    "hysteresis" : 5.0
  } ]
}


<Response [200]>

# Step 6: Check SMB health state

In [26]:
get("/nodes/cesNodes/health/states?filter=component=SMB")

curl -X GET -u admin:admin001 -k -H 'content-type: application/json' -H 'accepts: application/json' 'https://9.155.108.201/scalemgmt/v2/nodes/cesNodes/health/states?filter=component=SMB'
{
  "states" : [ {
    "activeSince" : "2019-03-20 21:03:45,317",
    "component" : "SMB",
    "entityName" : "os-13.novalocal",
    "entityType" : "NODE",
    "oid" : 8513,
    "reportingNode" : "os-13.novalocal",
    "state" : "HEALTHY"
  }, {
    "activeSince" : "2019-03-20 21:03:46,961",
    "component" : "SMB",
    "entityName" : "os-12.novalocal",
    "entityType" : "NODE",
    "oid" : 8520,
    "reportingNode" : "os-12.novalocal",
    "state" : "HEALTHY"
  } ],
  "status" : {
    "code" : 200,
    "message" : "The request finished successfully."
  }
}


<Response [200]>

# Step 7: Check threshold health state

In [25]:
get("/nodes/os-11/health/states?filter=component=THRESHOLD")

curl -X GET -u admin:admin001 -k -H 'content-type: application/json' -H 'accepts: application/json' 'https://9.155.108.201/scalemgmt/v2/nodes/os-11/health/states?filter=component=THRESHOLD'
{
  "states" : [ {
    "activeSince" : "2019-03-15 10:49:08,482",
    "component" : "THRESHOLD",
    "entityName" : "os-11.novalocal",
    "entityType" : "NODE",
    "oid" : 7875,
    "reportingNode" : "os-11.novalocal",
    "state" : "HEALTHY"
  }, {
    "activeSince" : "2019-03-15 10:49:25,058",
    "component" : "THRESHOLD",
    "entityName" : "op_time_custom",
    "entityType" : "THRESHOLD_RULE",
    "oid" : 7898,
    "reportingNode" : "os-11.novalocal",
    "state" : "HEALTHY"
  }, {
    "activeSince" : "2019-03-15 10:49:25,110",
    "component" : "THRESHOLD",
    "entityName" : "MemFree_Rule",
    "entityType" : "THRESHOLD_RULE",
    "oid" : 7899,
    "reportingNode" : "os-11.novalocal",
    "state" : "HEALTHY"
  } ],
  "status" : {
    "code" : 200,
    "message" : "The request finished succe

<Response [200]>

# Optional: Take a snapshot of the root fileset

In [14]:
snapshotName = "initialSnapshotFor_"  + filesetName
data = {
  "snapshotName": snapshotName
}

waitForJobResult(post("/filesystems/gpfs0/filesets/root/snapshots", data))
#waitForJobResult(delete("/filesystems/gpfs0/filesets/root/snapshots/" + snapshotName, data))

curl -X DELETE -u admin:admin001 -k -H 'content-type: application/json' -H 'accepts: application/json' 'https://9.155.108.201/scalemgmt/v2/filesystems/gpfs0/filesets/root/snapshots/initialSnapshotFor_newResearchFset'
{
  "jobs" : [ {
    "jobId" : 1000000000035,
    "status" : "RUNNING",
    "submitted" : "2019-03-21 08:35:20,819",
    "completed" : "N/A",
    "runtime" : 0,
    "request" : {
      "type" : "DELETE",
      "url" : "/scalemgmt/v2/filesystems/gpfs0/filesets/root/snapshots/initialSnapshotFor_newResearchFset"
    },
    "result" : { },
    "pids" : [ ]
  } ],
  "status" : {
    "code" : 202,
    "message" : "The request was accepted for processing."
  }
}
curl -X GET -u admin:admin001 -k -H 'content-type: application/json' -H 'accepts: application/json' 'https://9.155.108.201/scalemgmt/v2/jobs/1000000000035'
{
  "jobs" : [ {
    "jobId" : 1000000000035,
    "status" : "COMPLETED",
    "submitted" : "2019-03-21 08:35:20,819",
    "completed" : "2019-03-21 08:35:22,165",
   

(True, <Response [200]>)

# Optional: View executed mm-commands

In [5]:
get("/cliauditlog?fields=:all:")

curl -X GET -u admin:admin001 -k -H 'content-type: application/json' -H 'accepts: application/json' 'https://9.155.108.201/scalemgmt/v2/cliauditlog?fields=:all:'
{
  "auditLogRecords" : [ {
    "arguments" : "GUI_MGMT_SERVERS -N os-11",
    "command" : "mmcrnodeclass",
    "entryTime" : "2019-01-31T16:09:00",
    "exitTime" : "2019-01-31T16:09:01",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 7929,
    "returnCode" : 0,
    "user" : "root"
  }, {
    "arguments" : "GUI_SERVERS -N os-11,os-13,os-12",
    "command" : "mmcrnodeclass",
    "entryTime" : "2019-01-31T16:09:04",
    "exitTime" : "2019-01-31T16:09:05",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 9029,
    "returnCode" : 0,
    "user" : "root"
  }, {
    "arguments" : "GUI_CCR_CHANGE --command /usr/lpp/mmfs/gui/callbacks/global/ccrChangedCallback_421.sh --event ccrFileChange --async -N GUI_MGMT_SERVERS --parms root --object gui_master_node,_gui.user.repo,_gui.snapshots,_gui.not

    "exitTime" : "2019-02-05T19:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 13229,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-02-05T20:59:03",
    "exitTime" : "2019-02-05T20:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 28662,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-02-05T21:59:03",
    "exitTime" : "2019-02-05T21:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 12347,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-02-05T22:59:08",
    "exitTime" : "2019-02-05T22:59:09",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 28008,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all li

    "entryTime" : "2019-02-09T23:59:03",
    "exitTime" : "2019-02-09T23:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 4542,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-02-10T00:59:03",
    "exitTime" : "2019-02-10T00:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 20145,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-02-10T01:59:03",
    "exitTime" : "2019-02-10T01:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 3761,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-02-10T02:59:03",
    "exitTime" : "2019-02-10T02:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 19316,
    "returnCode" : 0,
    "user" : "S

    "entryTime" : "2019-02-14T03:59:03",
    "exitTime" : "2019-02-14T03:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 27776,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-02-14T04:59:03",
    "exitTime" : "2019-02-14T04:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 11335,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-02-14T05:59:03",
    "exitTime" : "2019-02-14T05:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 27192,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-02-14T06:59:03",
    "exitTime" : "2019-02-14T06:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 10905,
    "returnCode" : 0,
    "user" : 

    "command" : "mmaudit",
    "entryTime" : "2019-02-18T07:59:03",
    "exitTime" : "2019-02-18T07:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 19324,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-02-18T09:00:10",
    "exitTime" : "2019-02-18T09:00:11",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 2857,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-02-18T09:59:03",
    "exitTime" : "2019-02-18T09:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 18642,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-02-18T10:59:03",
    "exitTime" : "2019-02-18T10:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 2334,
    "retur

...

    "arguments" : "object_database_node replace -N os-12.novalocal",
    "command" : "mmchnodeclass",
    "entryTime" : "2019-02-25T09:53:22",
    "exitTime" : "2019-02-25T09:53:23",
    "node" : "os-12.novalocal",
    "originator" : "CLI",
    "pid" : 32678,
    "returnCode" : 0,
    "user" : "root"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-02-25T09:59:03",
    "exitTime" : "2019-02-25T09:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 4896,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-02-25T10:59:37",
    "exitTime" : "2019-02-25T10:59:38",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 29892,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-02-25T11:59:04",
    "exitTime" : "2019-02-25T11:59:05",
    "node" : 

    "arguments" : "object_singleton_node delete -N os-13.novalocal",
    "command" : "mmchnodeclass",
    "entryTime" : "2019-02-27T22:12:13",
    "exitTime" : "2019-02-27T22:12:14",
    "node" : "os-13.novalocal",
    "originator" : "CLI",
    "pid" : 13519,
    "returnCode" : 0,
    "user" : "root"
  }, {
    "arguments" : "object_database_node replace -N os-13.novalocal",
    "command" : "mmchnodeclass",
    "entryTime" : "2019-02-27T22:12:52",
    "exitTime" : "2019-02-27T22:12:53",
    "node" : "os-13.novalocal",
    "originator" : "CLI",
    "pid" : 19010,
    "returnCode" : 0,
    "user" : "root"
  }, {
    "arguments" : "object_singleton_node replace -N os-13.novalocal",
    "command" : "mmchnodeclass",
    "entryTime" : "2019-02-27T22:13:46",
    "exitTime" : "2019-02-27T22:13:47",
    "node" : "os-13.novalocal",
    "originator" : "CLI",
    "pid" : 22979,
    "returnCode" : 0,
    "user" : "root"
  }, {
    "arguments" : "object_database_node delete -N os-13.novalocal",
    

  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-03-03T12:59:03",
    "exitTime" : "2019-03-03T12:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 14037,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-03-03T13:59:03",
    "exitTime" : "2019-03-03T13:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 7324,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-03-03T14:59:03",
    "exitTime" : "2019-03-03T14:59:04",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 580,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-03-03T15:59:03",
    "exitTime" : "2019-03-03T15:59:04",
    "node" : "os-11.novalocal",
    "originator"

  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-03-07T14:59:06",
    "exitTime" : "2019-03-07T14:59:07",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 14452,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-03-07T15:59:04",
    "exitTime" : "2019-03-07T15:59:05",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 7294,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-03-07T16:59:12",
    "exitTime" : "2019-03-07T16:59:13",
    "node" : "os-11.novalocal",
    "originator" : "GUI",
    "pid" : 1178,
    "returnCode" : 0,
    "user" : "SYSTEM"
  }, {
    "arguments" : "all list -Y",
    "command" : "mmaudit",
    "entryTime" : "2019-03-07T17:59:03",
    "exitTime" : "2019-03-07T17:59:04",
    "node" : "os-11.novalocal",
    "originator

<Response [200]>